In [1]:
import pandas as pd
train_df = pd.read_csv("../sales_train.csv.gz")
test_df  = pd.read_csv("../test.csv.gz")

In [9]:
%store -r __prepare_data
__prepare_data

In [4]:
all_data = all_data[all_data.date_block_num > 0] # no lags before time 0
all_data.fillna(0, inplace=True)
print(all_data.isnull().values.any())

False


In [5]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [6]:
predictors = ["target_lag_1","item_target_enc_lag_1","shop_target_enc_lag_1"]
target = all_data.target.values
target[target>20] = 20
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1, random_state=321)

In [7]:
# models
sklRf  = RandomForestRegressor(n_estimators=8, max_features=1, n_jobs=2) #len(predictors)/2
# fit the model
sklRf.fit(all_data[predictors], target)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=1, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=8, n_jobs=2, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [24]:
%store -r __prepare_submission

IPython.macro.Macro(u'def prepare_submission(model, predictors, aliases, output):\n    """A helper to prepare sumbission file\n    Args:\n        model - anything implementing predict\n        predictors - ordered list of the predictors used for the model\n        aliases - dictionary of predictors of the model and their names in data\n        output - csv file name\n    """\n    # subset\n\n    X = all_data_33.loc[:,[aliases[colname] for colname in predictors]]\n    # rename\n    X.columns = predictors\n    # predict\n    y = sklRf.predict(X)\n\n    all_data_33["item_cnt_month"] = y\n    test = test_df.join(all_data_33.set_index(["shop_id","item_id"]), on=["shop_id","item_id"]).fillna(0)\n\n    test = test.loc[:,[\'ID\', \'item_cnt_month\']]\n    test.set_index("ID", inplace=True)\n    test["item_cnt_month"] = test["item_cnt_month"].round().astype("int64")\n\n    # clip\n    test[test.item_cnt_month>20] = 20\n    # save\n    test.to_csv(output)\n')

In [32]:
__prepare_submission

In [31]:
aliases = \
{"target_lag_1":          "target",
 "item_target_enc_lag_1": "item_target_enc",
 "shop_target_enc_lag_1": "shop_target_enc"}

prepare_submission(sklRf, predictors, aliases, "skl_rf_test.csv")

In [33]:
sklGbt = GradientBoostingRegressor(n_estimators=8, learning_rate=0.1, max_depth=8, random_state=0, loss='ls')

# fit the model
sklGbt.fit(all_data[predictors], target)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=8, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=8, n_iter_no_change=None, presort='auto',
             random_state=0, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [35]:
prepare_submission(sklGbt, predictors, aliases, "skl_gbt_test.csv")

In [36]:
sklRf  = RandomForestRegressor(n_estimators=32, max_features=1, n_jobs=2) #len(predictors)/2
sklRf.fit(all_data[predictors], target)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=1, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=32, n_jobs=2, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [37]:
prepare_submission(sklRf, predictors, aliases, "skl_rf_test2.csv")

In [38]:
sklGbt = GradientBoostingRegressor(n_estimators=32, learning_rate=0.1, max_depth=8, random_state=0, loss='ls')
sklGbt.fit(all_data[predictors], target)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=8, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=32, n_iter_no_change=None, presort='auto',
             random_state=0, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [39]:
prepare_submission(sklGbt, predictors, aliases, "skl_gbt_test2.csv")

In [40]:
sklRf  = RandomForestRegressor(n_estimators=128, max_features=1, n_jobs=2) #len(predictors)/2
sklRf.fit(all_data[predictors], target)
prepare_submission(sklRf, predictors, aliases, "skl_rf_test3.csv")

In [ ]:
#import math
#math.sqrt( mean_squared_error(testSet[['ptTrueInv']].values.ravel(), 1./sklRfTest)